In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\ANUSHA\\Desktop\\project'

In [5]:
os.chdir(r"C:\Users\ANUSHA\Desktop\project\dataset")

In [6]:
import nltk
import re
import numpy as np # linear algebra
import pandas as pd # data processing
import random
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize


#import pandas_profiling

%matplotlib inline

In [7]:
data=pd.read_csv("reviews.csv")

In [28]:
data

['enjoy vintag book movi enjoy read book plot unusu think kill someon self defens leav scene bodi without notifi polic hit someon jaw knock would wash today still good read',
 'book reissu old one author born era say nero wolf introduct quit interest explain author forgotten never heard languag littl date time like call gun heater also made good use fire dictionari look word like deshabil canarsi still well worth look see',
 'fairli interest read old style terminolog glad get read stori coars crasslanguag read fun relax like free ebooksbecaus check writer decid intrigu innov enough command englishthat convey stori without crude languag',
 'never read ami brewster mysteri one realli hook',
 'like period piec cloth lingo enjoy mysteri author guess least way',
 'beauti depth charact descript make like fast pace movi piti mr merwin write instead ami brewster mysteri',
 'enjoy one tho sure call ami brewster mysteri much clean well written charact well drawn',
 'never heard ami brewster need

In [8]:
data.head()

,id,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [9]:
df=data

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 10 columns):
id                1499 non-null int64
asin              1499 non-null object
helpful           1499 non-null object
overall           1499 non-null int64
reviewText        1499 non-null object
reviewTime        1499 non-null object
reviewerID        1499 non-null object
reviewerName      1495 non-null object
summary           1499 non-null object
unixReviewTime    1499 non-null int64
dtypes: int64(3), object(7)
memory usage: 117.2+ KB


In [11]:
df.describe()

,id,overall,unixReviewTime
count,1499.00000,1499.00000,1.499000e+03
mean,749.00000,4.12008,1.344308e+09
std,432.86834,1.10720,5.426525e+07
min,0.00000,1.00000,1.036454e+09
25%,374.50000,4.00000,1.312070e+09
50%,749.00000,4.00000,1.361578e+09
75%,1123.50000,5.00000,1.386418e+09
max,1498.00000,5.00000,1.405555e+09


In [14]:
df=df[['reviewText','overall']]
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
reviewText    1499 non-null object
overall       1499 non-null int64
dtypes: int64(1), object(1)
memory usage: 23.5+ KB


In [16]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 0 to 1498
Data columns (total 2 columns):
reviewText    1499 non-null object
overall       1499 non-null int64
dtypes: int64(1), object(1)
memory usage: 35.1+ KB


In [17]:
df=df[df['overall']!=3]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296 entries, 0 to 1498
Data columns (total 2 columns):
reviewText    1296 non-null object
overall       1296 non-null int64
dtypes: int64(1), object(1)
memory usage: 30.4+ KB


In [18]:
df=df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 2 columns):
reviewText    1296 non-null object
overall       1296 non-null int64
dtypes: int64(1), object(1)
memory usage: 20.4+ KB


In [19]:
df['sentiment']=np.where(df['overall']>3,1,0)
df.head()

,reviewText,overall,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,5,1
1,This book is a reissue of an old one; the auth...,4,1
2,This was a fairly interesting read. It had ol...,4,1
3,I'd never read any of the Amy Brewster mysteri...,5,1
4,"If you like period pieces - clothing, lingo, y...",4,1


In [20]:
df.tail()

,reviewText,overall,sentiment
1291,Why ruin a good book with sex scenes?This was ...,2,0
1292,"At first, I didn't know what to expect from th...",5,1
1293,The Remnants: Stories of the Jewish Resistance...,5,1
1294,I admit that at first I thought I was reading ...,1,0
1295,"No the usual holocaust book, although they are...",5,1


In [21]:
df

,reviewText,overall,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,5,1
1,This book is a reissue of an old one; the auth...,4,1
2,This was a fairly interesting read. It had ol...,4,1
3,I'd never read any of the Amy Brewster mysteri...,5,1
4,"If you like period pieces - clothing, lingo, y...",4,1
...,...,...,...
1291,Why ruin a good book with sex scenes?This was ...,2,0
1292,"At first, I didn't know what to expect from th...",5,1
1293,The Remnants: Stories of the Jewish Resistance...,5,1
1294,I admit that at first I thought I was reading ...,1,0


In [22]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANUSHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
data=[]
for i in range(0,1296):
    review=df["reviewText"][i]
    review=re.sub('[^a-zA-Z]',' ',review)
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    data.append(review)

In [31]:
data

['enjoy vintag book movi enjoy read book plot unusu think kill someon self defens leav scene bodi without notifi polic hit someon jaw knock would wash today still good read',
 'book reissu old one author born era say nero wolf introduct quit interest explain author forgotten never heard languag littl date time like call gun heater also made good use fire dictionari look word like deshabil canarsi still well worth look see',
 'fairli interest read old style terminolog glad get read stori coars crasslanguag read fun relax like free ebooksbecaus check writer decid intrigu innov enough command englishthat convey stori without crude languag',
 'never read ami brewster mysteri one realli hook',
 'like period piec cloth lingo enjoy mysteri author guess least way',
 'beauti depth charact descript make like fast pace movi piti mr merwin write instead ami brewster mysteri',
 'enjoy one tho sure call ami brewster mysteri much clean well written charact well drawn',
 'never heard ami brewster need

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
cv=CountVectorizer(max_features=1500)

In [55]:
X=cv.fit_transform(df['reviewText'])

In [56]:
X

<1296x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 65301 stored elements in Compressed Sparse Row format>

In [57]:
X.shape

(1296, 1500)

In [58]:
y=df['sentiment'].values

In [59]:
y.shape

(1296,)

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, random_state=0)

In [91]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
tf.compat.v1.get_default_graph
tf.random.uniform

<function tensorflow.python.ops.random_ops.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)>

In [92]:
model=Sequential()

In [121]:
model.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 1500))
model.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
model.add(Dense(output_dim = 1, init = 'uniform', activation = 'softmax'))

C:\Users\ANUSHA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1500, units=6, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.
C:\Users\ANUSHA\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
C:\Users\ANUSHA\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=1, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [122]:
model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [123]:
model.fit(X_train,y_train,batch_size=32,epochs=25)

Epoch 1/25
777/777 [==============================] - 1s 1ms/step - loss: 1.8671 - acc: 0.8829
Epoch 2/25
777/777 [==============================] - 0s 113us/step - loss: 1.8671 - acc: 0.8829
Epoch 3/25
777/777 [==============================] - 0s 122us/step - loss: 1.8671 - acc: 0.8829
Epoch 4/25
777/777 [==============================] - 0s 128us/step - loss: 1.8671 - acc: 0.8829
Epoch 5/25
777/777 [==============================] - 0s 127us/step - loss: 1.8671 - acc: 0.8829
Epoch 6/25
777/777 [==============================] - 0s 129us/step - loss: 1.8671 - acc: 0.8829
Epoch 7/25
777/777 [==============================] - 0s 120us/step - loss: 1.8671 - acc: 0.8829
Epoch 8/25
777/777 [==============================] - 0s 120us/step - loss: 1.8671 - acc: 0.8829
Epoch 9/25
777/777 [==============================] - 0s 114us/step - loss: 1.8671 - acc: 0.8829
Epoch 10/25
777/777 [==============================] - 0s 119us/step - loss: 1.8671 - acc: 0.8829
Epoch 11/25
777/777 [==========

In [125]:
y_pred=model.predict(X_test)
y_pred=y_pred>0.5

In [126]:
r=model.predict(cv.transform(["bad "]))

In [127]:
if (r>0.5):
    print("Positive")
else:
    print("Negative")

Positive
